In [27]:
# Max and Min points shifts. Localised max and min. Buys and sells above and below a certain %

In [28]:
# Cash is unlimited in both polarity with $10 per transaction

In [29]:
# Works excellent. Beyond my expectation.

In [30]:
!pip install websocket-client

In [31]:
import websocket
#import talib
import numpy as np
import json

In [32]:
cc = 'btcusdt'
interval = '1m'

socket = f'wss://stream.binance.com:9443/ws/{cc}@kline_{interval}'

In [33]:
amount_left = 10
# core_trade_amount = amount*0.80
# core_quantity = 0
# trade_amount = amount*0.20
#core_to_trade = True

portfolio = 0
investment, real_time_portfolio_value, closes, highs, lows = [], [], [], [], []
money_end = amount_left
lowest_pt = 1000000000000
highest_pt = 0

In [34]:
# Buying and Selling functions

def buy(allocated_money, price):
  global portfolio, money_end
  quantity = allocated_money/price
  money_end -= quantity*price
  portfolio += quantity
  if investment == []:
    investment.append(allocated_money)
  else:
    investment.append(allocated_money)
    investment[-1] += investment[-2]

def sell(allocated_money, price):
  global money_end, portfolio
  quantity = allocated_money / price
  money_end -= allocated_money
  portfolio -= quantity
  investment.append(-allocated_money)
  investment[-1] += investment[-2]

In [35]:
def on_message(ws,message):
    global portfolio, investment, closes, highs, lows, money_end, core_to_trade, core_quantity, real_time_portfolio_value
    global highest_pt, lowest_pt, amount_left
    json_message = json.loads(message)
    cs = json_message['k']
    candle_closed, close, high, low = cs['x'], cs['c'], cs['h'], cs['l']

    if candle_closed:
        closes.append(float(close))
        highs.append(float(high))
        lows.append(float(low))
        last_price = closes[-1]
        print(f'Closes: {closes}')
        if(last_price < lowest_pt):
            lowest_pt = last_price
        if(last_price > highest_pt):
            highest_pt = last_price
        print(f'Lowest_Point: {lowest_pt}')
        print(f'Highest_Point: {highest_pt}')
        
        if(last_price < closes[-4]):
            lowest_pt = last_price
        if(last_price > closes[-4]):
            highest_pt = last_price


        # If last price < 95% of Max price then sell
        if (last_price < highest_pt * 0.993):
            sell(-amount_left, price=last_price)
            print(f'We sold ${-amount_left} worth of bitcoin', '\n', '\n')  

        # If last price > 105% of Min price then buy   
        if (last_price > lowest_pt * 1.007):
            buy(amount_left, price=last_price)
            print(f'We bought ${amount_left} worth of bitcoin', '\n', '\n')
            
        print(money_end)
        print((highest_pt-lowest_pt)/last_price*100)
        print("\n")

#     if trade_amt > 0:
#       buy(trade_amt, price=last_price)
#       print(f'We bought ${trade_amt} worth of bitcoin', '\n', '\n')
#     elif trade_amt < 0:
#       sell(-trade_amt, price=last_price)
#       print(f'We sold ${-trade_amt} worth of bitcoin', '\n', '\n')
    
    
ws = websocket.WebSocketApp(socket, on_message=on_message)

In [ ]:
ws.run_forever()